In [1]:
# Import libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import time

In [2]:
# Read in project names to search
# Read in any list of project names from a csv. Directories hidden from code for privacy.
names = pd.read_csv(r'project_names.csv')
names_list = names['names'].tolist()

In [3]:
def bing_scrape(name):

    # Define query parameters and user agent
    query = name
    query = query.replace(' ', '+')
    URL = "https://www.bing.com/search?q=site%3Awww.ccgp.gov.cn+" + name
    USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)\
    Chrome/80.0.3987.87 Safari/537.36"

    # Request and get results

    headers = {"user-agent" : USER_AGENT}
    resp = requests.get(URL, headers=headers)

    if resp.status_code == 200:
        soup_test = BeautifulSoup(resp.content, "html.parser")

    links = []
    titles = []
    test = soup_test.findAll('h2')
    for title in test:
        titles.append(title.text.strip())
        links.append(title.find('a', attrs={'href': re.compile("^http://")}).replaceWithChildren()['href'])

    original_search = [query] * len(links)
    results = dict(zip(titles, zip(original_search, links)))
    
    return results

In [4]:
final = {}

for i in range(0, len(names_list)):
    name = names_list[i]
    final.update(bing_scrape(name))

In [7]:
test = pd.DataFrame(final).transpose()
test.reset_index(inplace=True)

In [26]:
test.rename(columns = {0: 'Original search'},inplace=True)
test.rename(columns = {1: 'Url'},inplace=True)
test.rename(columns = {'index': 'Title'},inplace=True)

In [27]:
test

,Title,Original search,Url
0,凌源市水务局凌源市自来水存量资产PPP项目公开招 …,凌源市自来水存量资产PPP项目,http://www.ccgp.gov.cn/cggg/dfgg/gkzb/201911/t...
1,PPP采购公告 - 中国政府采购网,黑龙江省大庆市林甸经济开发区污水处理厂PPP项目,http://www.ccgp.gov.cn/ppp/pppzbgg/index_2.html
2,PPP中标公告 - 中国政府采购网,黑龙江省绥化市经开区（国家级）污水处理工程项目,http://www.ccgp.gov.cn/ppp/pppzhbgg/index_24.html
3,海城市财政局海城市自来水存量资产PPP项目预中标 …,凌源市自来水存量资产PPP项目,http://www.ccgp.gov.cn/cggg/dfgg/zbgg/201912/t...
4,海城市财政局海城市自来水存量资产PPP项目资格预 …,凌源市自来水存量资产PPP项目,http://www.ccgp.gov.cn/cggg/dfgg/zbgg/201912/t...
...,...,...,...
1198,黑龙江省城镇污水处理项目打包招商 - ccgp.gov.cn,黑龙江省绥化市北郊污水处理工程PPP项目,http://www.ccgp.gov.cn/cggg/dfgg/qtgg/201904/t...
1199,望奎县人民医院污水处理设备采购项目竞争性磋商,黑龙江省绥化市北郊污水处理工程PPP项目,http://www.ccgp.gov.cn/cggg/dfgg/jzxcs/201908/...
1200,庆安县住房和城乡建设局庆安县污水处理厂提标升级 …,黑龙江省绥化市庆安县污水处理厂提标升级改造工程PPP项目,http://www.ccgp.gov.cn/cggg/dfgg/cjgg/201908/t...
1201,鸡西市梨树区城乡建设局黑龙江省鸡西市梨树区平岗 …,黑龙江省鸡西市梨树区平岗污水处理项目,http://www.ccgp.gov.cn/cggg/dfgg/zgysgg/201712...


In [28]:
# Write output to csv. Directory structure hidden in code for privacy.
test.to_csv(r'bing_scrape.csv', encoding='utf_8_sig')